In [4]:
import pandas as pd
import os
from datetime import datetime
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import plot_partial_dependence
from sklearn.inspection import partial_dependence 
from sklearn.metrics import classification_report


%matplotlib inline

In [2]:
nm = pd.read_pickle("data/all_calls17_uof_df.pkl")

# need to:
# make a y and make categories numeric(?)
# drop CAD and Event and Force
# make dummie columns
# cahnge month and day to integers


In [3]:
nm.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1459754 entries, 0 to 3986667
Data columns (total 14 columns):
CAD_Event_Number               1459754 non-null int64
Event_Clearance_Description    1459754 non-null object
Force_Level                    1459754 non-null object
Precinct                       1459754 non-null object
Sector                         1449873 non-null object
Beat                           1459754 non-null object
Subject_Race                   1459754 non-null object
Subject_Gender                 1459754 non-null object
Call_Type                      1459754 non-null object
Initial_Call_Type              1459754 non-null object
month                          1459754 non-null object
day                            1459754 non-null object
day_of_week                    1459754 non-null int64
hour                           1459754 non-null int64
dtypes: int64(3), object(11)
memory usage: 167.1+ MB


In [ ]:
y = nm2.Force_Level
y.replace({"Level 1 - Use of Force":1,"Level 2 - Use of Force":2,"Level 3 - OIS":4, "Level 3 - Use of Force": 3 })


In [ ]:
# drop columns and make time ints
nm2.drop(["Force_Level", "CAD_Event_Number", "Event_Clearance_Description"], axis=1, inplace=True)
nm2["month"] = nm2["month"].astype(int)
nm2["day"] = nm2["day"].astype(int)

In [ ]:
# better categories in Initial Call Type

init = nm2["Initial_Call_Type"]
to_drop = init[init.str.startswith("-ASSIGNED DUTY")]


s = nm2["Initial_Call_Type"].str.replace(" - .*", "", regex=True)
nm2["Initial_Call_Type"] = s

In [ ]:
# get dummies for string valued columns
nm2 = pd.get_dummies(data = nm2, 
                     columns=["Precinct", "Sector", "Beat", "Subject_Gender", "Subject_Race", "Call_Type", 
                               "Initial_Call_Type"], 
                     prefix=["P",  "S", "B", "SG", "SR","CT", "ICT"])

In [ ]:
# get column names to identify features in feature_importance later

cols = nm2.columns

In [ ]:
# function for later to categorize ICT column

def initial_call_type_cats(value):    
    if "MVC" in value:
        return "Motor Vehicle Collision"
    elif value.startswith("FRAUD"):
        return "Fraud"
    elif "Theft" in value:
        return "THEFT"
    elif "SUSPICIOUS STOP" in value:
        return value
    elif "SUSPICIOUS" in value:
        return "Suspicious person, vehicle, incident"
    elif "TRAFFIC STOP" in value:
        return value
    elif "ROAD RAGE" in value:
        return "Road Rage"
    elif "MOVING VIOLATION" in value:
        return "Moving Violation"
    elif "BLOCKING ROADWAY" in value:
        return "Blocking Roadway"
    elif "TRAFFIC" in value:
        return value
    elif "DISTURBANCE" in value:
        return "Disturbance"
    elif "OFF DUTY EMPLOYMENT" in value:
        return "OFF DUTY EMPLOYMENT"
    else:
        return value

In [ ]:
# calling above function to make list to make column
new_col = []
for idx, row in df.iterrows():
    value = row.col
    new_col.append(initial_call_type_cats(value))

In [ ]:
# try without this first - it took so long

X_scaled = StandardScaler().fit_transform(nm2)

In [ ]:
# will need to change nm2 to X_scaled if we use this later

X_train, X_test, y_train, y_test = train_test_split(nm2, y, test_size=0.2, random_state=659)

In [ ]:
# create RFC model - change all but random_state after we time this one

clf = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=659, oob_score=False)



In [ ]:
# fit the model

clf.fit(X_train, y_train)

In [ ]:
# check the feature importances and col_name list match first

len(clf.feature_importances_), len(cols)

In [ ]:
# most important features in predictions of this model

indices = np.argsort(clf.feature_importances_)
big = indices[-1:-20:-1]

cols[big]

In [ ]:
# least important features
little = indices[:10]
cols[little]


In [ ]:
clf.oob_score_ # accuracy of training set

In [ ]:
# predicted classes

yhat = clf.predict(X_test)

In [ ]:
sum(yhat == y_test) / len(yhat) # accuracy of testing set

In [ ]:
# sklearn.metrics.classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False)



In [5]:
# predict probabilities - what do you want threshold to be?

pp = clf.predict_proba(X_test)

NameError: name 'clf' is not defined

In [ ]:
# some rows' probabilities of being in each class:

for p in pp[:10]:
    print(f"Pr of Force 1 is: {p[0]}, Pr of Force 2: {p[1]}, Pr of Force 3: {p[3]}, & Pr of Officer Involved Shooting: {p[2]}")
          
          